In [13]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
from config import vworld_key
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [14]:
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [15]:
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [16]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [17]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]

### 격자(매핑용)

In [18]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [19]:
# crp_df에서 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
polygon_expanded = crp_df['centroid_polygon_geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# grid_map_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].intersects(polygon)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택, within 은 포함
# grid_map_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid_expanded = grid_map_df[grid_map_df['geometry'].intersects(polygon_expanded)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택

### 청주시 도시재생구역 거주인구 필터링 및 격자에 매핑

국제연합(UN)의 기준에 따르면 전체 인구에서 65세 이상이 차지하는 비율인 고령자 인구 비율이 7% 이상이면 고령화 사회, 14% 이상이면 고령 사회, 20% 이상이면 초고령 사회로 구분된다.

국제 연합 기준(UN)과 국내 국민연금 수급연령(연금 지급개시연령) 및 근로자의 법상 정년(노동관계법령상 정년(60세) 의무규정)을 고려하여 60대를 기준으로 고령화 비율을 나누었다. 

In [20]:
respop_df = pd.read_csv('SBJ_2309_001/1.청주시_거주인구.csv')

#원도심 영역 grid ID 리스트 와 매핑 & respop_df 필터링
grid_id = filtered_grid['properties.gid'].tolist()
filtered_respop = respop_df[respop_df['gid'].isin(grid_id)]
filtered_respop = filtered_respop.fillna(0)
filtered_respop = pd.concat([filtered_respop.iloc[:, :2], filtered_respop.iloc[:, 2:].astype(int)], axis=1)
filtered_respop.rename(columns = {"year": "연도"}, inplace = True)
filtered_respop['연도'] = filtered_respop['연도'].astype(str)

# 성별구분 없이 통합 및 세대별 인구수 데이터프레임으로 변환
for i in range(2, len(filtered_respop.columns), 2):
    col_name = f'{filtered_respop.columns[i][2:4]}대' if filtered_respop.columns[i][2] != '1' else f'{filtered_respop.columns[i][2:5]}대'
    filtered_respop[col_name] = filtered_respop.iloc[:, i:i+2].sum(axis=1)

# 인구 column & 고령인구 column & 고령인구 비율(%) column 생성
filtered_respop = filtered_respop[list(filtered_respop.columns[:2]) + list(filtered_respop.columns[20:])]
filtered_respop['인구'] = filtered_respop.iloc[:, 2:].sum(axis=1)
filtered_respop =  filtered_respop[filtered_respop['인구'] != 0] # 거주인구가 없는 격자 필터링(제외)
filtered_respop['고령인구'] = filtered_respop.iloc[:, 6:-1].sum(axis=1)
filtered_respop['고령인구_비율'] = filtered_respop.iloc[:, -1] / filtered_respop.iloc[:, -2]
filtered_respop = filtered_respop.fillna(0) # NaN값 0으로 대체
filtered_respop['고령인구_비율'] = filtered_respop['고령인구_비율'].apply(lambda x : round(x*100, 2))
filtered_respop

,gid,연도,20대,30대,40대,50대,60대,70대,80대,90대,100대,인구,고령인구,고령인구_비율
86,다바993480,2021,23,0,13,0,8,6,0,0,0,50,14,28.00
106,다바990496,2021,0,0,0,16,7,7,0,0,0,30,14,46.67
212,다바986484,2021,7,0,0,17,13,0,0,0,0,37,13,35.14
289,다바993479,2021,12,6,8,7,6,0,0,0,0,39,6,15.38
438,다바992492,2021,7,7,0,9,0,0,0,0,0,23,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394207,다바991476,2022,0,0,0,7,0,0,11,0,0,18,11,61.11
394529,다바992495,2022,10,0,0,0,15,0,0,0,0,25,15,60.00
394992,다바989477,2022,0,0,0,6,8,9,6,0,0,29,23,79.31
395475,다바991495,2022,10,0,0,15,0,0,0,0,0,25,0,0.00


#### 원도심지역 거주인구 연도별 변화(bar graph)

In [21]:
year_lst = range(2018, 2023, 1)
year_pop = [] # 인구수 종합
elder_pop = [] # 고령인구 종합
elder_rate = [] # 고령인구 비율
non_elder_pop = [] # 비고령인구 종합
non_elder_rate = [] # 비고령인구 비율
for year in year_lst:
    # 해당 연도의 인구수 종합
    year_pop.append(filtered_respop[filtered_respop['연도'] == str(year)]['인구'].sum())
    # 해당 연도의 고령인구 종합
    elder_pop.append(filtered_respop[filtered_respop['연도'] == str(year)]['고령인구'].sum())
    # 해당 연도의 고령인구 비율
    rate = filtered_respop[filtered_respop['연도'] == str(year)]['고령인구'].sum() / filtered_respop[filtered_respop['연도'] == str(year)]['인구'].sum()
    elder_rate.append(round(rate * 100, 2))
    # 해당 연도의 비고령인구 종합
    non_elder = filtered_respop[filtered_respop['연도'] == str(year)]['인구'].sum() - filtered_respop[filtered_respop['연도'] == str(year)]['고령인구'].sum()
    non_elder_pop.append(non_elder)
    # 해당 연도의 고령인구 비율
    rate = non_elder / filtered_respop[filtered_respop['연도'] == str(year)]['인구'].sum()
    non_elder_rate.append(round(rate * 100, 2))

In [22]:
fig = px.line(x=year_lst, y=year_pop, labels={'x':'연도', 'y':'인구수'}, title='연도별 인구 변화')
fig.update_traces(mode='lines+markers')
# 각 마커 위에 주석을 추가합니다.
for i in range(len(year_lst)):
    fig.add_annotation(
        x=year_lst[i],
        y=year_pop[i] + 130,
        text=f"{year_pop[i]}",
        showarrow=False,
    )
fig.update_xaxes(range=[2017, 2023])
fig.show()

In [23]:
fig = px.line(x=year_lst, y=elder_pop, labels={'x':'연도', 'y':'인구수'}, title='연도별 고령인구 변화')
fig.update_traces(mode='lines+markers')
# 각 마커 위에 주석을 추가합니다.
for i in range(len(year_lst)):
    fig.add_annotation(
        x=year_lst[i],
        y=elder_pop[i] + 40,
        text=f"{elder_pop[i]}",
        showarrow=False,
    )
fig.update_xaxes(range=[2017, 2023])
fig.show()

In [24]:
fig = px.line(x=year_lst, y=elder_rate, labels={'x':'연도', 'y':'인구비율(%)'}, title='연도별 고령인구 비율 변화')
fig.update_traces(mode='lines+markers')
# 각 마커 위에 주석을 추가합니다.
for i in range(len(year_lst)):
    fig.add_annotation(
        x=year_lst[i],
        y=elder_rate[i] + 0.3,
        text=f"{elder_rate[i]}",
        showarrow=False,
    )
fig.update_xaxes(range=[2017, 2023])
fig.show()

왜 2021년에 훅 떨어졌지...? 인구는 증가하는데 고령인구 비율이 3%가량 저하됐으면 비고령층 인구유입?

In [25]:
fig = px.line(x=year_lst, y=non_elder_pop, labels={'x':'연도', 'y':'인구수'}, title='연도별 비고령인구 비율 변화')
fig.update_traces(mode='lines+markers')
# 각 마커 위에 주석을 추가합니다.
for i in range(len(year_lst)):
    fig.add_annotation(
        x=year_lst[i],
        y=non_elder_pop[i] + 100,
        text=f"{non_elder_pop[i]}",
        showarrow=False,
    )
fig.update_xaxes(range=[2017, 2023])
fig.show()

In [26]:
fig = px.line(x=year_lst, y=non_elder_rate, labels={'x':'연도', 'y':'인구비율(%)'}, title='연도별 비고령인구 변화')
fig.update_traces(mode='lines+markers')
# 각 마커 위에 주석을 추가합니다.
for i in range(len(year_lst)):
    fig.add_annotation(
        x=year_lst[i],
        y=non_elder_rate[i] + 0.4,
        text=f"{non_elder_rate[i]}",
        showarrow=False,
    )
fig.update_xaxes(range=[2017, 2023])
fig.show()

In [27]:
# 격자 id : 격자 폴리곤 매핑 딕셔너리
map_dic = {}
for _, row in filtered_grid.iterrows():
    map_dic[row['properties.gid']] = row['geometry']

In [28]:
geometry_lst = []
for _, row in filtered_respop.iterrows():
    geometry_lst.append(map_dic[row['gid']])
filtered_respop['geometry'] = geometry_lst

#### 원도심 거주인주 고령화 정도 시각화(격자매핑) - 연도선택 가능 & 거주민 없는 지역 필터링 가능(격자 체크박스)

In [29]:
#  고령화 정도에 따라 grid색상을 지정하는 함수
def rate_color(rate):
    if rate > 20:
        return 'red'
    elif 30 > rate >= 14:
        return 'orange'
    elif 14 > rate >= 7:
        return 'yellow'
    elif 7 > rate >= 0:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 130px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 고령화 정도 범례 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 초고령 구역 (60세 이상인구 20% 이상)<br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 고령 구역 (60세 이상인구 20% 이상)<br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 고령화 구역 (60세 이상인구 14% 이상)<br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 일반 구역 (60세 이상인구 7% 이하)<br>
     &nbsp; <i style="background:white">&nbsp;</i>&nbsp; 거주민 없는 구역
     </div>
     """

# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2018 = folium.FeatureGroup(name="2018년", overlay=False) 
m_2019 = folium.FeatureGroup(name="2019년", overlay=False)
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)

year_lst = [str(year) for year in range(2018, 2023)]
layers_lst = [m_2018, m_2019, m_2020, m_2021, m_2022]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2018부터 2022까지의 연도
    filtered_respop_year = filtered_respop[filtered_respop['연도'] == year]

    # 'gid' 값을 기준으로 그룹화
    for gid in filtered_respop_year['gid'].unique():
        subset = filtered_respop_year[filtered_respop_year['gid'] == gid].copy()
        rate = subset['고령인구_비율'].iloc[0]
        color = rate_color(rate)
        subset['고령인구_비율'] = subset['고령인구_비율'].apply(lambda x: str(x) + '%')

        # subset_popup 데이터프레임을 HTML로 변환하여 popup text에 저장
        subset_popup = subset.iloc[:, 1:-1].style.hide(axis='index').set_properties(
            **{'border': '1px solid black'}).to_html()  # [['연도', '고령인구_비율']]
        popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{subset_popup}</div>"
        color = rate_color(rate)
        folium.GeoJson(
            filtered_respop_year[filtered_respop_year['gid'] == gid]['geometry'].iloc[0],
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 1}
        ).add_to(layers_dic[year]).add_child(folium.Popup(popup_text, max_width=1200), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

#### 원도심 거주인구 HEATMAP - 연도선택 가능

In [32]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2018 = folium.FeatureGroup(name="2018년", overlay=False) 
m_2019 = folium.FeatureGroup(name="2019년", overlay=False)
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)

year_lst = [str(year) for year in range(2018, 2023)]
layers_lst = [m_2018, m_2019, m_2020, m_2021, m_2022]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2018부터 2022까지의 연도
    subset = filtered_respop[filtered_respop['연도'] == year]
    # '인구'과 'geometry'을 사용하여 HeatMap 생성
    heat_data = [[point.centroid.xy[1][0], point.centroid.xy[0][0], aging] for point, aging in zip(subset['geometry'], subset['인구'])]
    HeatMap(heat_data, max_opacity=0.6, radius=15).add_to(layers_dic[year])

for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

#### 원도심 거주인구 고령화 정도 HEATMAP - 연도선택 가능

In [31]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2018 = folium.FeatureGroup(name="2018년", overlay=False) 
m_2019 = folium.FeatureGroup(name="2019년", overlay=False)
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)

year_lst = [str(year) for year in range(2018, 2023)]
layers_lst = [m_2018, m_2019, m_2020, m_2021, m_2022]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2018부터 2022까지의 연도
    subset = filtered_respop[filtered_respop['연도'] == year]
    # '고령인구_비율'과 'geometry'을 사용하여 HeatMap 생성
    heat_data = [[point.centroid.xy[1][0], point.centroid.xy[0][0], aging] for point, aging in zip(subset['geometry'], subset['고령인구_비율'])]
    HeatMap(heat_data, max_opacity=0.6, radius=15).add_to(layers_dic[year])

for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m